In [26]:
import chromadb
import pandas as pd

# Connect to your ChromaDB host
client = chromadb.HttpClient(host="170.64.231.135", port=8000)

client.list_collections()

[]

In [27]:

collection = client.get_collection(name="ghg_collection")

# Fetch all documents (adjust limit if needed)
results = collection.get(include=["documents", "metadatas", "embeddings"], limit = 1000)

# Convert to DataFrame
df = pd.DataFrame({
    "document": results["documents"],
    "metadata": results["metadatas"],
    "embedding": results["embeddings"]
})

# Save to CSV
df.to_csv("ghg_chunks_export.csv", index=False)

print("✅ Exported to ghg_chunks_export.csv")

Exception: {"error":"ValueError('Collection ghg_collection does not exist.')"}

In [12]:
import pandas as pd
chunk_text = pd.read_csv("ghg_chunks_export.csv")


chunk_text.head()

,document,metadata,embedding
0,Page 1:\nMain Content: COMPENDIUM\nOF GREENHOU...,"{'chunk_number': 1, 'page_count': 898, 'source...","[0.07204415649175644, 0.051991917192935944, 0...."
1,.................................................,"{'chunk_number': 10, 'page_count': 898, 'sourc...","[0.035045888274908066, 0.04470682516694069, 0...."
2,Greenhouse Gas Emissions Estimation and Invent...,"{'chunk_number': 100, 'page_count': 898, 'sour...","[0.03205292671918869, 0.03343049809336662, 0.0..."
3,"b Shires, T.M. Methane Emissions from the Natu...","{'chunk_number': 1000, 'page_count': 898, 'sou...","[0.023349491879343987, 0.019071441143751144, 0..."
4,"101, September 2004). This equates to 0.015 to...","{'chunk_number': 1001, 'page_count': 898, 'sou...","[0.024319730699062347, -0.004943027161061764, ..."


In [20]:
from groq import Client
import os

# Set API key
os.environ["GROQ_API_KEY"] = "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"
groq_client = Client(api_key=os.getenv("GROQ_API_KEY"))

# Build context from all chunks (filtered to stay under 4000 tokens)
chunk_column = "document" if "document" in chunk_text.columns else chunk_text.columns[0]
chunks = chunk_text[chunk_column].dropna().astype(str).tolist()

# Keep appending chunks until reaching character limit (~16,000 chars = ~4k tokens)
max_chars = 16000
selected_chunks = []
current_len = 0

for c in chunks:
    if current_len + len(c) > max_chars:
        break
    selected_chunks.append(c)
    current_len += len(c)

filtered_context = "\n---\n".join(selected_chunks)

# Prompt
prompt = f"""
You are a GHG emissions consultant and expert in international and Australian climate reporting frameworks.

Using the full context below, generate a list of **question and answer (Q&A) pairs** that summarize key insights, regulatory obligations, and challenges for companies.

Context:
{filtered_context}

Format the output like this:
Q1: ...
A1: ...
Q2: ...
A2: ...
(...continue...)

Only include the question–answer pairs in your response.

Begin:
"""

# Query Groq
chat_completion = groq_client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a knowledgeable climate consultant generating clear and accurate Q&A pairs."},
        {"role": "user", "content": prompt}
    ]
)

# Show result
response = chat_completion.choices[0].message.content
print("🧠 Generated Q&A Pairs:\n")
print(response)

🧠 Generated Q&A Pairs:

Here are the Q&A pairs summarizing key insights, regulatory obligations, and challenges for companies:

Q1: What is the primary purpose of the Compendium of Greenhouse Gas Emissions Methodologies for the Natural Gas and Oil Industry?
A1: The compendium provides a comprehensive guide for companies in the natural gas and oil industry to measure and report their greenhouse gas (GHG) emissions.

Q2: Who developed the compendium, and who sponsored the work?
A2: The compendium was developed by ERM and sponsored by the American Petroleum Institute (API), with leadership and direction by Marcus Koblitz, Corporate Policy.

Q3: What is the scope of the compendium in terms of industry coverage?
A3: The compendium specifically focuses on the natural gas and oil industry, covering emissions methodologies relevant to this sector.

Q4: How can companies in the natural gas and oil industry benefit from using this compendium?
A4: Companies can use the compendium to ensure accura